In [1]:
from train import *
import matplotlib.pyplot as plt

image_size = 512

c:\Users\JW\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data = LabelledImageDataset('dataset/val', transform = A.Compose([
        A.Resize(image_size, image_size),   
        A.Normalize()
    ]))
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

val_data = LabelledImageDataset('dataset/val', transform = A.Compose([
        A.Resize(image_size, image_size),   
        A.Normalize()
    ]))
val_loader = DataLoader(val_data, batch_size=128, shuffle=False)

In [ ]:
# visualize
def display_img(img,label):
    print(f"Label: {'primary' if label else 'footway'}")
    plt.imshow(img.permute(1,2,0))

train_features, train_labels = next(iter(val_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")


In [ ]:
for i, l in zip(train_features, train_labels):
    #plt.imshow(i.numpy().transpose(1,2,0))
    plt.title(l)
    img = i.squeeze()
    plt.imshow(img.permute(1, 2, 0))
    plt.show()
    print(f"Label: {'primary' if l else 'footway'}")

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

def show_batch(dl):
    """Plot images grid of single batch"""
    for images, labels in dl:
        fig,ax = plt.subplots(figsize = (16,12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images,nrow=16).permute(1,2,0))
        break
        
show_batch(val_loader)

In [3]:
num_epochs = 30
opt_func = torch.optim.Adam
lr = 0.001

model = HighwayClassifier()

#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)